In [10]:
### PACKAGES
import csv
import numpy as np
import cv2
import os
import matplotlib
import matplotlib.pyplot as plt
from tensorflow.keras import layers
import pickle
from copy import deepcopy
from asift import *
#import keras
from tensorflow import keras #for kelvin's machine
from find_obj import init_feature, filter_matches, explore_match


In [11]:
### CONSTANTS
ROOTPATH = './'
IMG_HEIGHT = 490
IMG_WIDTH = 680
NUM_CHANNELS = 3

In [12]:
train_img_names = [] # list of train image names as strings
train_img_labels = [] # list of train images labels as tuples (latitude, longitude)
with open('COMP90086_2021_Project_train/train.csv') as train_csv:
    reader = csv.reader(train_csv, delimiter=',')
    next(reader) # skip header row
    for row in reader:
        name = row[0] # string
        label = (float(row[1]), float(row[2])) # tuple
        train_img_names.append(name)
        train_img_labels.append(label)

# train images stored as numpy array
resize = 0.5
IMG_HEIGHT = int(IMG_HEIGHT * resize)
IMG_WIDTH = int(IMG_WIDTH * resize)
train_size = (len(train_img_names), IMG_HEIGHT, IMG_WIDTH,NUM_CHANNELS )
#print(train_images.shape)


train_images = np.zeros(train_size, dtype='uint8')
for i in range(len(train_img_names)):
    name = train_img_names[i]
    subpath = 'COMP90086_2021_Project_train/train/' + name + '.jpg'
    img = cv2.imread(os.path.join(ROOTPATH, subpath))
    img = cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))
#     print(img.shape)
    train_images[i] = img

# train labels stored as numpy array
train_labels = np.array(train_img_labels)
print(train_labels[0])
print(train_images.shape)


[-9.38067808  3.58271965]
(7500, 245, 340, 3)


In [13]:
test_img_names = [] # list of test image names as strings
with open('COMP90086_2021_Project_test/imagenames.csv') as test_csv:
    reader = csv.reader(test_csv, delimiter=',')
    next(reader) # skip header row
    for row in reader:
        name = row[0] # string
        test_img_names.append(name)
        
# test images stored as numpy array
test_size = (len(test_img_names), IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS)
test_images = np.zeros(test_size, dtype='uint8')
for i in range(len(test_img_names)):
    name = test_img_names[i]
    subpath = 'COMP90086_2021_Project_test/test/' + name + '.jpg'
    img = cv2.imread(os.path.join(ROOTPATH, subpath))
    img = cv2.resize(img,(IMG_WIDTH,IMG_HEIGHT))

    
    test_images[i] = img
    
# train_images = train_images.reshape(train_images.shape[0], 32, 32, 3).astype('float32')
# test_images = test_images.reshape(test_images.shape[0], 32, 32, 3).astype('float32')


In [33]:
##CHECK
train_img_1 = cv2.cvtColor(train_images[0], cv2.COLOR_BGR2RGB)
train_img_n = cv2.cvtColor(train_images[-1], cv2.COLOR_BGR2RGB)
test_img_1 = cv2.cvtColor(test_images[0], cv2.COLOR_BGR2RGB)
print(train_img_1.shape)
test_img_n = cv2.cvtColor(test_images[-1], cv2.COLOR_BGR2RGB)

plt.subplot(2,2,1)
plt.imshow(train_img_1) 
plt.title('train image 1')
plt.axis('off')
plt.subplot(2,2,2)
plt.imshow(train_img_n) 
plt.title('train image n')
plt.axis('off')
plt.subplot(2,2,3)
plt.imshow(test_img_1) 
plt.title('test image 1')
plt.axis('off')
plt.subplot(2,2,4)
plt.imshow(test_img_n) 
plt.title('test image n')
plt.axis('off')
plt.show()

NameError: name 'train_images' is not defined

In [60]:
# FIRST ATTEMPT: BASELINE 
model = keras.models.Sequential()

model.add(layers.Input((IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS)))
model.add(layers.Conv2D(32, (3, 3), activation='relu'))
model.add(layers.Conv2D(16, (3, 3), activation='relu'))
model.add(layers.Dense(16, activation='relu'))
model.add(layers.Dense(2))


#op = keras.optimizers.adam_v2.Adam()
op = keras.optimizers.Adam() # for kelvin's machine
model.compile(loss='mae', optimizer=op)
print(train_images.shape)
model.summary()

h = model.fit(train_images, train_labels,verbose = 1,batch_size=1,validation_split = 0.1,epochs = 20)

(7500, 245, 340, 3)
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 243, 338, 32)      896       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 241, 336, 16)      4624      
_________________________________________________________________
dense (Dense)                (None, 241, 336, 16)      272       
_________________________________________________________________
dense_1 (Dense)              (None, 241, 336, 2)       34        
Total params: 5,826
Trainable params: 5,826
Non-trainable params: 0
_________________________________________________________________
Train on 6750 samples, validate on 750 samples
Epoch 1/20
6750/6750 [==============================] - 1154s 171ms/sample - loss: 41.2145 - val_loss: 38.8998
Epoch 2/20
4915/6750 [====================>.........] - ETA: 5:15 - loss: 

KeyboardInterrupt: 

In [8]:
import sys
def sizeof_fmt(num, suffix='B'):
    ''' by Fred Cirera,  https://stackoverflow.com/a/1094933/1870254, modified'''
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f %s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f %s%s" % (num, 'Yi', suffix)

for name, size in sorted(((name, sys.getsizeof(value)) for name, value in locals().items()),
                         key= lambda x: -x[1])[:10]:
    print("{:>30}: {:>8}".format(name, sizeof_fmt(size)))

     test_images_descriptors_1:  1.4 GiB
                   test_images: 286.0 MiB
                           img: 244.2 KiB
                  train_labels: 117.3 KiB
            descriptors_labels: 116.1 KiB
               gray_test_image: 81.5 KiB
               train_img_names: 65.6 KiB
              train_img_labels: 65.6 KiB
                           pad: 36.1 KiB
                test_img_names:  9.8 KiB


In [ ]:
k_nn = 2
lowe_ratio = 0.7
num_closest_matches = 5

# FLANN parameters
FLANN_INDEX_KDTREE = 1
index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
search_params = dict(checks=50)   # or pass empty dictionary

# If there exists a checkpoint, use saved weights and indices
if os.path.exists('./checkpoint/weights.pickle') and os.path.exists('./checkpoint/indices.pickle'):
    with open(os.path.join("checkpoint","weights.pickle"),'rb') as handle:
        weights = pickle.load(handle)
    with open(os.path.join(os.getcwd(),"checkpoint","indices.pickle"),'rb') as handle:
        indices = pickle.load(handle)
        
    # Skip already completed test images
    test_images_sift = test_images[len(weights):]
    print("Found checkpoint, skipping to test image",len(weights))
else:
    weights = []
    indices = []
    


for i in range(len(test_images_sift)):    
    print(i)

    test_image = test_images_sift[i]
    # convert test image to grayscale
    gray_test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.SIFT_create() # initialise SIFT detector
    
    # SIFT keypoints and descriptors for test image
    keypoints_test, descriptors_test = sift.detectAndCompute(gray_test_image, None)
        
    # initialise FLANN
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    
    # stores 5 highest number of keypoint matches
    best_num_matches = np.zeros(num_closest_matches, dtype='uint64')
    
    matches_index = np.zeros(num_closest_matches,dtype='uint64')
    # stores 5 train images with highest number of keypoint matches
    #best_image_matches = np.zeros((num_closest_matches, IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS), dtype='uint8')
    # stores  corresponding matching images 
    #circles_and_lines = np.zeros((num_closest_matches, IMG_HEIGHT, IMG_WIDTH*2, NUM_CHANNELS), dtype='uint8')

    for j in range(len(train_images)):
        train_image = train_images[j]
        # convert train image to grayscale
        gray_train_image = cv2.cvtColor(train_image,cv2.COLOR_BGR2GRAY)
        
        # SIFT keypoints and descriptors for train image
        keypoints_train, descriptors_train = sift.detectAndCompute(gray_train_image, None)
        
        
        
        # when there is no descriptor less than k_nn descriptors
        if descriptors_test is None or descriptors_train is None or \
        descriptors_test.shape[0] < k_nn or descriptors_train.shape[0] < k_nn:
            continue
            
        # Draws a circle with the size of each keypoint and show its orientation
        keypoints_test_with_size = cv2.drawKeypoints(gray_test_image, keypoints_test, None,
                                                     flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        keypoints_train_with_size = cv2.drawKeypoints(gray_train_image, keypoints_train, None,
                                                      flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        
        # Matching descriptors using KNN algorithm
        matches = flann.knnMatch(descriptors_test, descriptors_train, k=k_nn)
        
        # Create a mask to draw all good matches
        matchesMask = []
        # Store all good matches as per Lowe's Ratio test.
        good = []
        for m,n in matches:
            if m.distance < lowe_ratio *n.distance:
                good.append(m)
                matchesMask.append([1,0]) # Match
            else:
                matchesMask.append([0,0]) # Mismatch
        num_matches = len(good)
        
        # Draw all good matches
        draw_params = dict(#matchColor = (0,255,0),  #If you want a specific colour
                           #singlePointColor = (255,0,0), #If you want a specific colour
                            matchesMask = matchesMask,
                            flags = cv2.DrawMatchesFlags_DEFAULT)

        good_matches = cv2.drawMatchesKnn(gray_test_image, keypoints_test, gray_train_image,
                                          keypoints_train, matches, None, **draw_params)
        
        # compares to closest matches and update as necessary
        for k in range(num_closest_matches):
            if num_matches >= best_num_matches[k]:
                best_num_matches = np.insert(best_num_matches, k, num_matches, 0)
                best_num_matches = np.delete(best_num_matches, -1, 0)
                matches_index = np.insert(matches_index, k, j, 0)
                matches_index = np.delete(matches_index, -1, 0)
                #best_image_matches = np.insert(best_image_matches, k, train_image, 0)
                #best_image_matches = np.delete(best_image_matches, -1, 0)
                #circles_and_lines = np.insert(circles_and_lines, k, good_matches, 0)
                #circles_and_lines = np.delete(circles_and_lines, -1, 0)
                break
    
    '''
    # display 5 closest images
    for z in range(num_closest_matches):
        plt.subplots(figsize=(10, 10)) 
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB), cmap='gray')  
        plt.title('Test Image')
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(best_image_matches[z], cv2.COLOR_BGR2RGB), cmap='gray')  
        plt.title('Train Image')
        plt.axis('off')
        plt.show()
        
        plt.figure(figsize=(15, 15))
        plt.imshow(circles_and_lines[z])
        plt.title(f'Number of good matches between two images: {best_num_matches[z]}')
        plt.axis('off')
        plt.show()
    '''
    weights.append(list(best_num_matches))
    indices.append(list(matches_index))
    
    if i % 5 == 0: # Save weights and Indices every 5 test images 
        with open(os.path.join("checkpoint","weights.pickle"),'wb') as handle:
            pickle.dump(weights, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
        with open(os.path.join(os.getcwd(),"checkpoint","indices.pickle"),'wb') as handle:
            pickle.dump(indices, handle, protocol=pickle.HIGHEST_PROTOCOL)
            

Found checkpoint, skipping to test image 315
0
1
2
3
4
5
6
7
8
9
10
11
12


In [ ]:
predictions_top1 = []
predictions_avg = []
predictions_weighted_avg = []
for i in range(len(indices)):
    weight = weights[i]
    test_img_indices = indices[i]
    
    ### PREDICTIONS_TOP_1
    predictions_top1.append(train_img_labels[test_img_indices[0]])
    
    ### PREDICTIONS_AVG
    pred_avg_x = 0
    pred_avg_y = 0
    for ind in test_img_indices:
        x_lab, y_lab= train_img_labels[ind]
        pred_avg_x += x_lab
        pred_avg_y += y_lab
    pred_avg_x /= len(test_img_indices)
    pred_avg_y /= len(test_img_indices)
    
    predictions_avg.append((pred_avg_x, pred_avg_y))
    
    ### PREDICTIONS_WEIGHTED_AVG
    pred_weighted_avg_x = 0
    pred_weighted_avg_y =0
    for j in range(len(test_img_indices)):
        x_lab, y_lab= train_img_labels[test_img_indices[j]]
        w = weight[j] / sum(weight)
        pred_weighted_avg_x += x_lab * w
        pred_weighted_avg_y += y_lab * w
    
    predictions_weighted_avg.append((pred_weighted_avg_x, pred_weighted_avg_y))

In [ ]:
header = ['id', 'x', 'y']
with open(ROOTPATH + 'predictions_top1.csv', 'w') as predictions_top1:
    writer = csv.writer(predictions_top1)
    writer.writerow(header)
    for i in range(len(test_img_names)):
        img_name = test_img_names[i]
        x_val, y_val = predictions_top1[i]
        data = [img_name, str(x_val), str(y_val)]
        writer.writerow(data)
    predictions_top1.close()
    
with open(ROOTPATH + 'predictions_avg.csv', 'w') as predictions_avg:
    writer = csv.writer(predictions_avg)
    writer.writerow(header)
    for i in range(len(test_img_names)):
        img_name = test_img_names[i]
        x_val, y_val = predictions_avg[i]
        data = [img_name, str(x_val), str(y_val)]
        writer.writerow(data)
    predictions_avg.close()
    
with open(ROOTPATH + 'predictions_weighted_avg.csv', 'w') as predictions_weighted_avg:
    writer = csv.writer(predictions_weighted_avg)
    writer.writerow(header)
    for i in range(len(test_img_names)):
        img_name = test_img_names[i]
        x_val, y_val = predictions_weighted_avg[i]
        data = [img_name, str(x_val), str(y_val)]
        writer.writerow(data)
    predictions_weighted_avg.close()

In [6]:
test_images_descriptors = np.zeros((len(train_images),200,128))
print(test_images_descriptors.shape)
rows_to_delete = []
descriptors_labels = deepcopy(train_labels)
for i in range(len(train_images)):    

    image = train_images[i]
    
    # convert test image to grayscale
    gray_test_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    sift = cv2.SIFT_create(200) # initialise SIFT detector
    
    # SIFT keypoints and descriptors for test image
    keypoints_test, descriptors_test = sift.detectAndCompute(gray_test_image, None)
    if descriptors_test is None:
        rows_to_delete.append(i)
        continue
    if descriptors_test.shape[0] < 200:
        times_to_pad =200-descriptors_test.shape[0]
        pad = np.zeros((times_to_pad,128))
        descriptors_test = np.r_[descriptors_test, pad]
    elif descriptors_test.shape[0] > 200:
        descriptors_test = descriptors_test[:200]
    test_images_descriptors[i] = descriptors_test
#     print(descriptors_test.shape)

print(descriptors_labels.shape)
 
if len(rows_to_delete) != 0:
    rows_to_delete.reverse()
    for ind in rows_to_delete:
        test_images_descriptors = np.delete(test_images_descriptors,ind,0)

        descriptors_labels =  np.delete(descriptors_labels,ind,0)
        
        
print(test_images_descriptors.shape)
print(descriptors_labels.shape)

        

    
    
    

(7500, 200, 128)
(7500, 2)
(7425, 200, 128)
(7425, 2)


In [13]:
# Attempt: CNN Using SIFT Descriptors as Features
test_images_descriptors_1 = np.expand_dims(test_images_descriptors, axis=-1)
test_images_descriptors_1 = test_images_descriptors_1/255

model = keras.models.Sequential()
model.add(layers.Input((200,128,1)))

# model.add(layers.Conv2D(16, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))
# model.add(layers.Conv2D(8, (3, 3), activation='relu'))
# model.add(layers.MaxPooling2D((2,2)))
model.add(layers.Flatten())
model.add(layers.Dense(50, activation='relu'))
model.add(layers.Dense(2))

op = keras.optimizers.Adam() # for kelvin's machine
model.compile(loss='mae', optimizer=op)
h = model.fit(test_images_descriptors_1, descriptors_labels,verbose = 1,batch_size=1,validation_split = 0.1,epochs = 10)


(7425, 200, 128, 1)
Epoch 1/10
6682/6682 [==============================] - 10s 2ms/step - loss: 42.1933 - val_loss: 40.0653
Epoch 2/10
6682/6682 [==============================] - 12s 2ms/step - loss: 38.6744 - val_loss: 39.9075
Epoch 3/10
6682/6682 [==============================] - 11s 2ms/step - loss: 34.4092 - val_loss: 40.4195
Epoch 4/10
6682/6682 [==============================] - 10s 1ms/step - loss: 30.1579 - val_loss: 41.1308
Epoch 5/10
6682/6682 [==============================] - 10s 1ms/step - loss: 26.9439 - val_loss: 42.1084
Epoch 6/10
6682/6682 [==============================] - 10s 1ms/step - loss: 23.9024 - val_loss: 42.5988
Epoch 7/10
6682/6682 [==============================] - 10s 1ms/step - loss: 21.6561 - val_loss: 42.5149
Epoch 8/10
6682/6682 [==============================] - 10s 1ms/step - loss: 19.6124 - val_loss: 43.8759
Epoch 9/10
6682/6682 [==============================] - 10s 1ms/step - loss: 17.9400 - val_loss: 43.7559
Epoch 10/10
6682/6682 [============

In [9]:
train_images1 = cv2.cvtColor(train_images[31], cv2.COLOR_BGR2GRAY)


detector, matcher = init_feature("sift-flann")

pool=ThreadPool(processes = cv.getNumberOfCPUs())

for j in range(len(train_images)):
    train_image = train_images[j]
    train_images2 = cv2.cvtColor(train_image, cv.COLOR_BGR2GRAY)

    kp1, desc1 = affine_detect(detector, train_images1, pool=pool)
    kp2, desc2 = affine_detect(detector, train_images2, pool=pool)

    if len(desc1) == 0 or len(desc2) == 0:
        continue
    raw_matches = matcher.knnMatch(desc1, trainDescriptors = desc2, k = 2) #2

    p1, p2, kp_pairs = filter_matches(kp1, kp2, raw_matches)
    H, status = cv.findHomography(p1, p2, cv.RANSAC, 5.0)
    print('%d / %d  inliers/matched' % (np.sum(status), len(status)))




<function affine_detect at 0x000002A60B13A430>
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
affine sampling: 43 / 43
aff

KeyboardInterrupt: 

In [ ]:

# Initialize asift with flann (Note: remove -flann to use bruteforce)
detector, matcher = init_feature("sift-flann")
pool=ThreadPool(processes = cv.getNumberOfCPUs())

# If there exists a checkpoint, use saved weights and indices
if os.path.exists('./checkpoint/asift_weights.pickle') and os.path.exists('./checkpoint/asift_indices.pickle'):
    with open(os.path.join("checkpoint","asift_weights.pickle"),'rb') as handle:
        weights = pickle.load(handle)
    with open(os.path.join(os.getcwd(),"checkpoint","asift_indices.pickle"),'rb') as handle:
        indices = pickle.load(handle)
        
    # Skip already completed test images
    test_images_asift=test_images[len(weights):]
    print("Found checkpoint, skipping to test image",len(weights))
else:
    weights = []
    indices = []
    

for i in range(len(test_images_asift)):    
    print(i)
    test_image = test_images_asift[i]
    # convert test image to grayscale
    gray_test_image = cv2.cvtColor(test_image, cv2.COLOR_BGR2GRAY)
    kp1, desc1 = affine_detect(detector, gray_test_image, pool=pool)

    
    # stores 5 highest number of keypoint matches
    best_num_matches = np.zeros(num_closest_matches, dtype='uint64')
    
    matches_index = np.zeros(num_closest_matches,dtype='uint64')
    # stores 5 train images with highest number of keypoint matches
    #best_image_matches = np.zeros((num_closest_matches, IMG_HEIGHT, IMG_WIDTH, NUM_CHANNELS), dtype='uint8')
    # stores  corresponding matching images 
    #circles_and_lines = np.zeros((num_closest_matches, IMG_HEIGHT, IMG_WIDTH*2, NUM_CHANNELS), dtype='uint8')

    for j in range(len(train_images)):
        train_image = train_images[j]
        # convert train image to grayscale
        gray_train_image = cv2.cvtColor(train_image,cv2.COLOR_BGR2GRAY)
        
        kp2, desc2 = affine_detect(detector, gray_train_image, pool=pool)

        raw_matches = matcher.knnMatch(desc1, trainDescriptors = desc2, k = 2) #2

        p1, p2, kp_pairs = filter_matches(kp1, kp2, raw_matches)
        H, status = cv.findHomography(p1, p2, cv.RANSAC, 5.0)
        print('%d / %d  inliers/matched' % (np.sum(status), len(status)))

    

        if len(desc1) == 0 or len(desc2) == 0:
            continue
        
    
        
#         # when there is no descriptor less than k_nn descriptors
#         if descriptors_test is None or descriptors_train is None or \
#         descriptors_test.shape[0] < k_nn or descriptors_train.shape[0] < k_nn:
#             continue
            
    
#         # Draws a circle with the size of each keypoint and show its orientation
#         keypoints_test_with_size = cv2.drawKeypoints(gray_test_image, keypoints_test, None,
#                                                      flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
#         keypoints_train_with_size = cv2.drawKeypoints(gray_train_image, keypoints_train, None,
#                                                       flags=cv2.DRAW_MATCHES_FLAGS_DRAW_RICH_KEYPOINTS)
        
        # Matching descriptors using KNN algorithm
        matches = flann.knnMatch(descriptors_test, descriptors_train, k=k_nn)
        
        # Create a mask to draw all good matches
        matchesMask = []
        # Store all good matches as per Lowe's Ratio test.
        good = []
        for m,n in matches:
            if m.distance < lowe_ratio *n.distance:
                good.append(m)
                matchesMask.append([1,0]) # Match
            else:
                matchesMask.append([0,0]) # Mismatch
        num_matches = len(good)
        
        # Draw all good matches
        draw_params = dict(#matchColor = (0,255,0),  #If you want a specific colour
                           #singlePointColor = (255,0,0), #If you want a specific colour
                            matchesMask = matchesMask,
                            flags = cv2.DrawMatchesFlags_DEFAULT)

        good_matches = cv2.drawMatchesKnn(gray_test_image, keypoints_test, gray_train_image,
                                          keypoints_train, matches, None, **draw_params)
        
        # compares to closest matches and update as necessary
        for k in range(num_closest_matches):
            if num_matches >= best_num_matches[k]:
                best_num_matches = np.insert(best_num_matches, k, num_matches, 0)
                best_num_matches = np.delete(best_num_matches, -1, 0)
                matches_index = np.insert(matches_index, k, j, 0)
                matches_index = np.delete(matches_index, -1, 0)
                #best_image_matches = np.insert(best_image_matches, k, train_image, 0)
                #best_image_matches = np.delete(best_image_matches, -1, 0)
                #circles_and_lines = np.insert(circles_and_lines, k, good_matches, 0)
                #circles_and_lines = np.delete(circles_and_lines, -1, 0)
                break
    
    '''
    # display 5 closest images
    for z in range(num_closest_matches):
        plt.subplots(figsize=(10, 10)) 
        plt.subplot(1, 2, 1)
        plt.imshow(cv2.cvtColor(test_image, cv2.COLOR_BGR2RGB), cmap='gray')  
        plt.title('Test Image')
        plt.axis('off')
        plt.subplot(1, 2, 2)
        plt.imshow(cv2.cvtColor(best_image_matches[z], cv2.COLOR_BGR2RGB), cmap='gray')  
        plt.title('Train Image')
        plt.axis('off')
        plt.show()
        
        plt.figure(figsize=(15, 15))
        plt.imshow(circles_and_lines[z])
        plt.title(f'Number of good matches between two images: {best_num_matches[z]}')
        plt.axis('off')
        plt.show()
    '''
    weights.append(list(best_num_matches))
    indices.append(list(matches_index))
    
    if i % 5 == 0: # Save weights and Indices every 5 test images 
        with open(os.path.join("checkpoint","asift_weights.pickle"),'wb') as handle:
            pickle.dump(weights, handle, protocol=pickle.HIGHEST_PROTOCOL)
            
        with open(os.path.join(os.getcwd(),"checkpoint","asift_indices.pickle"),'wb') as handle:
            pickle.dump(indices, handle, protocol=pickle.HIGHEST_PROTOCOL)
            